# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame

# file path
city_data_df = pd.read_csv("output_data/cities.csv")

# display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,stanley,54.8680,-1.6985,278.31,80,20,4.12,GB,1706288827
1,1,qaqortoq,60.7167,-46.0333,256.19,64,1,6.50,GL,1706288827
2,2,abalak,58.1286,68.5944,261.68,87,98,1.68,RU,1706288828
3,3,olonkinbyen,70.9221,-8.7187,271.09,74,88,7.98,SJ,1706288828
4,4,iqaluit,63.7506,-68.5145,247.42,68,40,8.75,CA,1706288554


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [13]:
%%capture --no-display

# Configure the map plot
humidity_plot_map = city_data_df.hvplot.points(x='Lng', y='Lat', size='Humidity', hover_cols=['City'],
                                                geo=True, colorbar=True, alpha=0.5, cmap='Blues', width=800, height=500,tiles='OSM')

# Display the map
humidity_plot_map


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_data_df = city_data_df[(city_data_df['Max Temp'] >= 295) & (city_data_df['Max Temp'] <= 305) &
                                  (city_data_df['Wind Speed']<= 10) & (city_data_df['Humidity'] <= 75)]

# Drop any rows with null values
cleaned_weather_df = ideal_weather_data_df.dropna()

# Display sample data
cleaned_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,bredasdorp,-34.5322,20.0403,296.79,63,31,5.14,ZA,1706288828
10,10,acarau,-2.8856,-40.1200,303.25,64,97,6.71,BR,1706288829
11,11,hadibu,12.6500,54.0333,298.55,72,73,1.46,YE,1706288829
21,21,utrik,11.2278,169.8474,298.70,61,80,3.59,MH,1706288831
23,23,callao,-12.0667,-77.1500,301.31,57,0,8.23,PE,1706288701


In [6]:
# shape
cleaned_weather_df.shape

(122, 10)

### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = cleaned_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

C:\Users\atlas\AppData\Local\Temp\ipykernel_49988\2657731280.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ''


,City,Country,Lat,Lng,Humidity,Hotel Name
7,bredasdorp,ZA,-34.5322,20.0403,63,
10,acarau,BR,-2.8856,-40.1200,64,
11,hadibu,YE,12.6500,54.0333,72,
21,utrik,MH,11.2278,169.8474,61,
23,callao,PE,-12.0667,-77.1500,57,


In [9]:
# shape
hotel_df.shape

(122, 6)

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
# Set parameters to search for a hotel
radius = 10000
params = {
   "text": "hotels",
   "categories": "accommodation",
   "radius": 10000,
   "lat": 0 ,
   "lon": 0,
   "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # update params lat and long
    params["lat"] = latitude
    params["lon"] = longitude
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bredasdorp - nearest hotel: Victoria Hotel
acarau - nearest hotel: No hotel found
hadibu - nearest hotel: No hotel found
utrik - nearest hotel: No hotel found
callao - nearest hotel: Casa Píos
key largo - nearest hotel: Key Largo Inn
mulayjah - nearest hotel: No hotel found
lodja - nearest hotel: No hotel found
puerto san jose - nearest hotel: Martita
saint peter - nearest hotel: Ancient Grove
robertsport - nearest hotel: Nana's Lodge
sawakin - nearest hotel: Hotel Halfa
jwaneng - nearest hotel: Mokala Lodge & Teemane Casino
tidjikja - nearest hotel: Auberge
touros - nearest hotel: Pousada Atlântico
dan gora - nearest hotel: No hotel found
kingston - nearest hotel: Aphrodite Hotel
dzilam gonzalez - nearest hotel: No hotel found
al fqih ben calah - nearest hotel: Hotel Oum Elfadl
gizo - nearest hotel: Rekona Lodge
mushie - nearest hotel: hôtel Mushi
punta arenas - nearest hotel: No hotel found
san luis de la loma - nearest hotel: No hotel found
cuddalore - nearest 

,City,Country,Lat,Lng,Humidity,Hotel Name
7,bredasdorp,ZA,-34.5322,20.0403,63,Victoria Hotel
10,acarau,BR,-2.8856,-40.1200,64,No hotel found
11,hadibu,YE,12.6500,54.0333,72,No hotel found
21,utrik,MH,11.2278,169.8474,61,No hotel found
23,callao,PE,-12.0667,-77.1500,57,Casa Píos
...,...,...,...,...,...,...
542,camayenne,GN,9.5350,-13.6878,60,Palm Camayenne
552,dhidhdhoo,MV,6.8833,73.1000,74,137
573,makokou,GA,0.5738,12.8642,44,No hotel found
574,koulikoro,ML,14.0000,-7.7500,9,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [18]:
%%capture --no-display

# Configure the map plot
hotel_plot_map = hotel_df.hvplot.points(
    x='Lng',
    y='Lat',
    hover_cols=['Hotel Name', 'Country'],
    geo=True,
    color='blue',
    size=20,
    width=800,
    height=500,
    tiles='OSM'
)
# Display the map
hotel_plot_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Country)